## Computer Representation of Real Numbers $\mathbb{R}$

*Corresponds to Section 1.2 of the lecture notes.  This notebook goes well beyond the material required for the course - including some tricky use of Python.  Mainly for motivated students.*

There are infinitely many real numbers, and infinitely many in the interval $[0,1]$ <a href=http://en.wikipedia.org/wiki/Real_number>[1]</a>.  In order to represent them in a (finite) computer, we have to encode them in a finite manner.  Representing all $x\in\mathbb{R}$ is not possible, and an imprefect approximation must be made.  We can notice this occationally when working with Python:

In [ ]:
3./10 - 1./10 - 1./10 - 1./10

Clearly the answer should be $0$.  Why is it not $0$? And why is it specifically this number?  Investigating further:

In [ ]:
print(repr(3./10))

In [ ]:
print(repr(1./10 + 1./10 + 1./10))

Which doesn't even seem to agree with the first result above...  [<b>Python note</b>: the "repr()" is necessary to prevent Python's "print" beautifying the output, and cutting off the trailing "4".  Check the effect of removing the "repr()".]  Python is using the almost universal <i>double-precision floating-point</i> arithmetic.  But first we have to consider how a computer can represent integers.

### Computer Integers

There are infinitely many integers, so again only a subset can be represented in a finite computer.  Thanks to physics and historical accident, modern computers are binary and use multiples of 8-bit <i>bytes</i> (although there might have been better alternatives <a href=http://web.williams.edu/Mathematics/sjmiller/public_html/105Sp10/addcomments/Hayes_ThirdBase.htm>[2]</a>).  I.e. data is represented internally as (e.g.):
$$
00101001\quad 01011000\quad 11101011 ...
$$
There are $2^8$ different possible states of a byte.  If an integer $z$ is represented with a byte we can represent

In [ ]:
2**8

different integers, e.g. from $0$ to $255$, or from $-128$ to $127$, or even every 2nd integer: $0,\,2,\,4,\dots,510$.  The choice of <i>encoding</i> is entirely ours.  Now $255$ is not a particularly large number.

**Exercise 1:** 

**(a) How can we increase the number of integers it is possible to represent?**

**(b) What do you think is a acceptable maximum integer for most applications?  Engineering?  Finance?  Astronomy? Chemistry?  Compare this with what you might know about the number of bits used in practice.  Are you sure it'll be enough? <a href=http://arstechnica.com/business/2014/12/gangnam-style-overflows-int_max-forces-youtube-to-go-64-bit/>[3]</a>**

**(c) Why do you think the $0,\,2,\,4,\dots,510$ system described above is not popular?**

If the $N$ bits are denoted
$$
b = (b_0, b_1, \cdots, b_{N-1})
$$
each taking values $0$ or $1$, then the standard encoding for positive integers $z$ is
$$
z = \sum_{i=0}^{N-1} b_i \cdot 2^i.
$$

**(d) What is the minimum/maximum value of $z$?  Can every integer between the minimum and maximum be represnted by some $b$?**

#### Python specifics

Python is almost unique amongst programming languages for having fully integrated "bignum" support, which Python calls "long".  When an integer grows too large, the interpreter automatically switches it to long, increasing the amount of bits used to represent it as far as necessary.  E.g.:

In [ ]:
from math import factorial
factorial(100)

However, almost everywhere else in the computing world the number of bits for repesenting integers is fixed.  To reproduce this behaviour in Python we can use numpy, which allows us to specify the data type when we define the array.  E.g. for an 8-bit (1 byte) integer:

In [ ]:
import numpy as np
z = np.zeros(1, dtype='int8')
print(repr(z))

#### Rounding behaviour

So what happens when we exceed the maximum?

In [ ]:
z = np.zeros(1, dtype='uint8')
z[0] = 0
z += 2**8-1
print(z)
z += 1
print(z)
z -= 1
print(z)

#### Looking at the memory (advanced)

With a bit of Python hacking, we can look inside numpy numbers, at the binary representation used in the memory of the computer:

In [ ]:
6*16 + 4

In [ ]:
from binascii import b2a_hex
def hex2binstr(hex_str): 
    table = {'0':'0000','1':'0001','2':'0010','3':'0011',
             '4':'0100','5':'0101','6':'0110','7':'0111', 
             '8':'1000','9':'1001','a':'1010','b':'1011',
             'c':'1100','d':'1101','e':'1110','f':'1111'}
    return  ' '.join([table[c] for c in hex_str.decode("utf-8")])

z = np.zeros(1, dtype='uint8')
                            ### Implementation details: 
z[0] = 1                    ### (a) The "[0]" is important, "z=1" would make "z" an "int" 
                            ###     rather than changing the value in the numpy array
hex = b2a_hex(z.data)       ### (b) "z.data" gives access to the raw data of "z"
                            ### (c) "b2a_hex()" converts that to a string with characters 
                            ###     0-9 a-f, each character representing 4 bits "hexadecimal"
bin = hex2binstr(hex)       ### (d) "hex2binstr()" takes the hexadecimal representation and 
                            ###     converts it to a 0s and 1s.
print('Number:',z[0])
print('Hex:   ',hex)
print('Binary:',bin)

The code above uses the 'uint8' datatype, unsigned 8-bit integer.  Try 'int8', signed (+ve and -ve) 8-bit integers.  There are also 'int16', 'uint16', 'int32', 'uint32', 'int64', 'uint64'.  To look at the memory representation of floats use 'float16', 'float32', 'float64'. 

### Real numbers: Fixed-point arithmetic

Given integers, a trivial way of representing real numbers is to use a constant interval $h<<1$:
$$
r = h \cdot \sum_{i=0}^{N-1} b_i \cdot 2^i.
$$
E.g. with 32-bits, and reserving 1 bit to represent the sign (+/-), with an interval $h$ of $1\times 10^{-4}$, we can represent numbers between $\pm 2^{31} \cdot 1\times 10^{-4} \approx \pm 200000$ with a resolution of $0.0001$.  Both range and accuracy are obviously very limited, and this can lead to all sorts of errors <a href=http://ta.twi.tudelft.nl/users/vuik/wi211/disasters.html>[4]</a>.


The smallest number $>0$ in this system is $h$.  And the smallest number $>1$ is $1+h$.  The main problem with this number system is that the distribution of representable numbers is uniform.  This means that there are the same number of numbers in the interval $[0,1]$ as in $[100000,100001]$, and in every other interval of width $1$.  Even though we likely need much more accuracy in $[0,1]$ than in $[100000,100001]$.  The solution is to use an exponential distribution of representable numbers - *floating point* arithmetic.

### Real numbers: Floating-point arithmetic

The "resolution" limitiations of fixed-point arithmetic can be solved by using two integers to represent a real number.  Given a <i>base</i> $b$ (usually $b=2$ or $b=10$):
$$
r = s \times b^e
$$
where the exponent $E_\mathrm{min} \leq e \leq E_\mathrm{max}$ and the significand $1 \leq s < b$ with $n$-digits.
For example, $b=10$, 5-digit significand $1.0000\leq s \leq 9.9999$, $-8\leq e\leq 8$.  Then
$10\cdot\pi = 3.1416\times 10^1$.  The system as described does not contain zero, this is usually added explicitly, and one bit controls the sign.

Returning to the two intervals $[0,1]$ and $[100000,100001]$, there is only on representable number in the latter (namely the number $100000$), but about $100000 \times 8$ representable numbers in the former.  The absolute spacing between numbers increases as the numbers increase, whereas the relative spacing remains roughly constant.


#### The IEEE 754 Standard

The floating-point representation of real numbers is universal in modern hardware.  The IEEE 754 standard <a href="http://en.wikipedia.org/wiki/IEEE_754-1985">[5]</a>  describes the exact form of the representation - not only which bits represent the exponent and significand, but also rounding rules, special numbers <tt>inf</tt>, <tt>-inf</tt> and <tt>nan</tt> "not-a-number".  We say no more about this, other than that the code above in "Looking at the memory" allows us to see the bit-wise representation of floats, by creating the numpy array with "<tt>dtype=float32</tt>" (single-precision) or "<tt>dtype=float64</tt>" (double-precision).

#### Decimal floating point in Python

To play with different floating-point number systems, the "decimal" module is nice.  It implements floating point numbers in base $b=10$, with user-specified significant digits and exponent range.  For example we can work with the simple example arithmetic defined above, 5-digit significand $1.0000\leq s \leq 9.9999$, $-8\leq e\leq 8$, as follows:

In [ ]:
from decimal import *
getcontext().prec = 5    # set significand digits to 5
getcontext().Emin = -8   # set min/max exponent to +-8
getcontext().Emax = 8
Decimal(np.pi)*10

Note that the precision rules are only enforced when a <tt>Decimal</tt> object is operated upon (by e.g. multiplication), so:

In [ ]:
Decimal(1./7)

But:

In [ ]:
Decimal(1) / Decimal(7)

**Exercise 2:**

**(a) Try to discover <i>by experiment</i> what the machine-epsilon is in this system (the smallest number which when added to $1$ gives a number other than $1$).  Is this consistent with what you've learned from the notes?**

### Rounding errors in Floating-point arithmetic

Finally we give a practical example of limitations of floating-point.  We evaluate a derivative numerically for $f(x)=e^x$ at $x=1$:
$$
\frac{df}{dx}(x) \approx \frac{f(x+h) -f(x)}{h} = \frac{\mathrm{exp}(1+h) -\mathrm{exp}(1)}{h}
$$

The step-size $h$ must be chosen small enough that <i>truncation error</i> is not significant - but is there also a lower limit on $h$?  What is it?  Experimenting:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

h = 10.**np.linspace(-16,1,18)
dapprox = (np.exp(1.+h) - np.exp(1.)) /h
dexact = np.exp(1.)
plt.plot(np.log10(h), np.log10(np.abs(dapprox - dexact)), '-o')
plt.xlabel(r'$log_{10}(h)$', fontsize=20)
plt.ylabel(r'$log_{10}(e)$', fontsize=20)

**(b) Interpret the plot - explain the increase in error at both large and small values of $h$.  What is the best choice of $h$?**